In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn import svm

In [2]:
data_url = "https://raw.githubusercontent.com/darkreqnx/leaf-classifier/master/img_dataset.csv?token=AIOJSRNXUP63AVJTN7BI67267RU5I"
df = pd.read_csv(data_url)

In [3]:
df.head(5)

,code,smoothness0,leaflet_desc0,leaflet_desc1,leaflet_desc2,leaflet_desc3,leaflet_desc4,leaflet_desc5,leaflet_desc6,gabor0,gabor1,gabor2,gabor3,gabor4,gabor5,gabor6,gabor7,color0,color1,color2,color3,color4,color5,color6,color7,color8,color9,color10,color11,fft0,fft1,fft2,fft3,fft4,fft5,fft6,fft7,fft8,fft9,fft10,...,geom11,geom12,geom13,geom14,geom15,geom16,geom17,geom18,geom19,geom20,geom21,geom22,geom23,geom24,geom25,geom26,geom27,geom28,geom29,geom30,geom31,geom32,geom33,geom34,geom35,geom36,geom37,geom38,geom39,geom40,geom41,geom42,geom43,geom44,geom45,geom46,geom47,geom48,geom49,geom50
0,0,0.990136,0.293172,5.898140,57.140699,38.832976,239.002092,182.043951,2736.354694,0.767506,0.803687,0.760917,0.733909,0.755205,0.782215,0.795676,0.751683,0.015092,0.005299,0.844378,0.133894,0.000138,0.000211,0.000815,0.000173,0.0,0.000000,0.000000,0,0.070427,0.068695,0.060660,0.049750,0.137163,0.117566,0.008226,0.037063,0.054877,0.036925,0.037894,...,-1.071620,-0.752859,0.611270,-1.097490,0.551494,0.196806,0.154064,0.018376,0.384700,0.095523,0.985284,1.666760,0.908266,1.055867,-1.011291,-0.457053,0.770790,-0.277475,-0.682188,0.551631,-0.746709,-0.328224,0.357519,-0.364019,-0.518048,-0.023039,-0.797469,0.996792,1.399421,-0.012732,0.267327,-1.018589,-0.007033,0.138642,0.389550,0.097027,0.441639,1.124257,0.695187,0.624561
1,0,0.947406,0.337538,6.429441,53.765871,50.358713,207.696895,151.489274,1581.968056,0.941950,0.847207,0.815962,0.808848,0.779815,0.818916,0.846362,0.917206,0.018860,0.012823,0.452758,0.515107,0.000215,0.000000,0.000049,0.000000,0.0,0.000042,0.000147,0,0.081972,0.083511,0.072986,0.054633,0.083010,0.127735,0.045627,0.010247,0.040070,0.048555,0.046960,...,-0.898505,-0.421081,0.889103,-0.878074,0.980123,-0.873983,0.199895,0.110950,0.600093,-0.317701,-0.085417,1.302360,0.220882,0.155951,-1.067050,-0.486090,0.724310,0.610906,-0.156947,-0.667784,-0.189414,-0.328224,-0.551312,0.301024,0.162199,-0.466444,-0.562133,-0.672715,-0.855704,-0.638822,1.355466,-0.573917,-0.396909,-0.366315,0.564329,1.121807,1.548557,1.111618,0.726782,0.565695
2,0,0.960641,0.266033,7.058012,62.578195,53.338541,242.886805,179.513231,2376.196104,0.898163,0.830881,0.805372,0.770342,0.755695,0.769563,0.822595,0.918369,0.014501,0.004747,0.550366,0.430199,0.000166,0.000000,0.000010,0.000000,0.0,0.000000,0.000010,0,0.067323,0.060668,0.053945,0.035244,0.105082,0.127844,0.030847,0.033383,0.066364,0.046371,0.020601,...,-1.047215,-0.670810,0.572735,-0.816224,0.704392,0.078168,-0.196897,0.056299,0.243914,0.248556,1.244132,1.569838,1.067666,1.281215,-1.047111,-0.515518,0.550105,0.045615,-0.687638,0.224539,-0.746709,-0.328224,-0.551312,-0.644368,-0.240554,0.040023,-0.886653,-0.734647,-0.712324,-0.526449,0.841678,-0.103730,-0.243852,-0.452981,0.032249,0.043864,0.544354,1.198798,0.831943,-0.459434
3,0,0.956420,0.227015,4.864612,48.330907,44.045431,216.004630,163.878003,1510.848786,0.797747,0.774420,0.658974,0.650082,0.651573,0.650955,0.668357,0.715309,0.019888,0.004747,0.575858,0.399218,0.000115,0.000000,0.000000,0.000000,0.0,0.000000,0.000175,0,0.093941,0.069459,0.080111,0.020528,0.116983,0.105256,0.050896,0.045107,0.043655,0.060127,0.031633,...,-1.017493,-0.604512,0.830339,-0.953665,1.416105,-0.409719,0.117211,-0.023152,-0.224811,0.568606,0.984145,0.804790,0.607204,-0.341478,-1.037871,-0.475758,0.748974,0.062967,-0.388759,-0.667784,-0.169367,-0.328224,-0.026609,0.020068,0.035388,-0.311361,-0.492448,-0.415298,-0.511806,-0.529879,0.944517,-0.553011,-0.119947,-0.107208,0.073492,0.719025,1.621169,1.107195,0.751325,0.751414
4,0,0.945802,0.229219,5.487981,49.724115,42.011903,179.446371,134.766465,1346.737691,1.057724,0.933728,0.704337,0.651289,0.659790,0.686938,0.714581,0.865413,0.025128,0.008429,0.363056,0.603132,0.000019,0.000037,0.000054,0.000000,0.0,0.000000,0.000146,0,0.089514,0.096912,0.073890,0.043355,0.044485,0.107758,0.090597,0.062963,0.019032,0.027247,0.058747,...,-0.929655,-0.293354,1.549450,-0.998710,2.369302,-1.318394,1.607098,0.153506,0.399876,-0.012222,1.111127

In [4]:
def util_vanilla(base_dataset):
  # stratified train-test split
  # exluding single valued class
  idx_single_val = base_dataset[base_dataset['code'] == 35].index[0]
  trial_dataset = base_dataset.drop([idx_single_val])
  # stratified train-test split
  train, test = train_test_split(trial_dataset, test_size=0.3, stratify=trial_dataset['code'], random_state=0)
  # reintroduce excluded value in train, split into X,y
  train = train.append(base_dataset.iloc[idx_single_val])
  # random train-test split
  # train, test = train_test_split(trial_dataset, test_size=0.3, random_state=0)
  X_train = train.drop(['code'], axis=1)
  y_train = train['code']
  X_test = test.drop(['code'], axis=1)
  y_test = test['code']
  return X_train, y_train, X_test, y_test

In [5]:
X_train, y_train, X_test, y_test = util_vanilla(df)

In [6]:
# blind test - across kernels linear, poly, rbf, sigmoid with defaults params

for kern in ['linear', 'poly', 'rbf', 'sigmoid']:
  globals()[kern+"_svm_clf"] = svm.SVC(kernel=kern)
  globals()[kern+"_svm_clf"].fit(X_train, y_train)
  globals()["y_pred_svm_"+kern] = globals()[kern+"_svm_clf"].predict(X_test)
  print(kern+"_svm classifier:", accuracy_score(y_test, globals()["y_pred_svm_"+kern]))

linear_svm classifier: 0.8398967075532602
poly_svm classifier: 0.11943189154293092
rbf_svm classifier: 0.14783731439638476
sigmoid_svm classifier: 0.04325371207230471


In [7]:
# standardized, and then rerun with defaults
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

for kern in ['linear', 'poly', 'rbf', 'sigmoid']:
  globals()[kern+"_svm_clf"] = svm.SVC(kernel=kern)
  globals()[kern+"_svm_clf"].fit(X_train_scaled, y_train)
  globals()["y_pred_svm_"+kern] = globals()[kern+"_svm_clf"].predict(X_test_scaled)
  print(kern+"_svm classifier:", accuracy_score(y_test, globals()["y_pred_svm_"+kern]))

linear_svm classifier: 0.908327953518399
poly_svm classifier: 0.6849580374435119
rbf_svm classifier: 0.8728211749515816
sigmoid_svm classifier: 0.7979341510652034


In [10]:
# GridSearchCV for SVM

param_grid = {'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],'kernel': ['linear', 'rbf', 'poly', 'sigmoid']}
grid = GridSearchCV(svm.SVC(),param_grid,refit=True,verbose=3)
grid.fit(X_train_scaled,y_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV] C=0.1, gamma=1, kernel=linear ...................................


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.906, total=   1.8s
[CV] C=0.1, gamma=1, kernel=linear ...................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.902, total=   1.8s
[CV] C=0.1, gamma=1, kernel=linear ...................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.6s remaining:    0.0s


[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.902, total=   1.8s
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.903, total=   1.8s
[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV] ....... C=0.1, gamma=1, kernel=linear, score=0.900, total=   1.8s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.037, total=   6.5s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.037, total=   6.5s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.037, total=   6.5s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.039, total=   6.4s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .

[Parallel(n_jobs=1)]: Done 320 out of 320 | elapsed: 16.3min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [11]:
# print best parameter after tuning 
print(grid.best_params_) 
  
# print how our model looks after hyper-parameter tuning 
print(grid.best_estimator_) 

{'C': 0.1, 'gamma': 1, 'kernel': 'linear'}
SVC(C=0.1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [12]:
grid_predictions = grid.predict(X_test_scaled) 
  
# print classification report 
print(classification_report(y_test, grid_predictions)) 

              precision    recall  f1-score   support

           0       1.00      0.94      0.97        17
           1       0.95      0.90      0.93        21
           2       0.93      1.00      0.96        13
           3       1.00      0.90      0.95        10
           4       0.89      0.89      0.89         9
           5       0.50      0.50      0.50         4
           6       0.67      0.50      0.57         4
           7       0.55      0.86      0.67         7
           8       0.85      0.73      0.79        15
           9       1.00      1.00      1.00        13
          10       0.86      0.86      0.86         7
          11       1.00      1.00      1.00         6
          12       0.82      0.95      0.88        19
          13       0.79      1.00      0.88        22
          14       0.00      0.00      0.00         1
          15       1.00      1.00      1.00         5
          16       0.94      0.94      0.94        17
          17       0.96    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [13]:
accuracy_score(y_test, grid_predictions)

0.907036797934151

In [18]:
for f in ['X_train', 'y_train', 'X_test', 'y_test']:
  globals()[f].to_csv("svm_"+f+".csv", index=0)

for nd in ['X_train_scaled', 'y_train_scaled', 'X_test_scaled', 'y_test_scaled']:
  np.savetxt("svm_"+nd+".csv",np.asarray(globals()[f]), delimiter=",")